In [1]:
from utils.read_video import read_video
from utils.BallDetector import BallDetector
from utils.CourtDetector import CourtDetector
from utils.PlayerDetector import PersonDetector
from utils.BounceDetector import BounceDetector
from utils.combine import combine
from utils.write import write_video, write_image
from utils.scene_manager import scene_detect

video_name = "5s.mp4"
video_path = f"test_videos/{video_name}"
output_path = f"results/output"

# Read video
frames, fps, original_width, original_height = read_video(video_path, resize=False)

# Scene detection
scenes = scene_detect(video_path)

#  Inference ball track
ball_detector = BallDetector(path_model='models/ball_track.pt',
                             original_width=original_width, 
                             original_height=original_height)
ball_track = ball_detector.infer_model(frames)

# Inference court
court_detector = CourtDetector(path_model='models/court_detector.pt',
                               original_width=original_width,
                               original_height=original_height)
homography_matrices, kps_court = court_detector.infer_model(frames)

# Inference player
person_detector = PersonDetector('cuda')
persons_top, persons_bottom = person_detector.track_players(frames, homography_matrices, filter_players=False)

# Inference bounce
bounce_detector = BounceDetector(path_model='models/bounce_detector.cbm')
x_ball = [x[0] for x in ball_track]
y_ball = [x[1] for x in ball_track]
bounces = bounce_detector.predict(x_ball, y_ball)

c:\Users\abiyamf\anaconda3\envs\courtplay\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
VideoManager is deprecated and will be removed.
`base_timecode` argument is deprecated and has no effect.
100%|██████████| 151/151 [00:12<00:00, 12.28it/s]


In [2]:
import cv2
import numpy as np
from dataclasses import dataclass
from typing import Generator
from utils.court_reference import CourtReference


MINIMAP_WIDTH = 166
MINIMAP_HEIGHT = 350
PLAYER_HEAT_INCREMENT = 5
PLAYER_HEAT_PERCENTILE = 98  # percentile that should map near red
PLAYER_HEAT_TARGET_DECAY = 0.985  # smoothing for dynamic scaling
PLAYER_HEAT_GAUSSIAN_SIGMA = 25
PLAYER_HEAT_RADIUS = 10
BALL_HEAT_INCREMENT = 12
BALL_HEAT_PERCENTILE = 96
BALL_HEAT_TARGET_DECAY = 0.99
BALL_HEAT_GAUSSIAN_SIGMA = 20
BALL_HEAT_RADIUS = 14
HEAT_ALPHA = 0.65
CONTOUR_LEVELS = [40, 90, 140, 190, 230]
PERCENTILE_TARGET_SAMPLES = 250_000


@dataclass
class CombineFrameOutputs:
    combined: np.ndarray
    minimap_ball: np.ndarray
    minimap_player: np.ndarray
    heatmap_player: np.ndarray
    heatmap_ball: np.ndarray


def get_court_img():
    court_reference = CourtReference()
    court_img = court_reference.court.copy()
    court_img = cv2.dilate(court_img, np.ones((10, 10), dtype=np.uint8))
    return court_img


def _draw_ball(frame, ball_track, index, draw_trace, trace_len):
    if not ball_track[index][0]:
        return frame
    if draw_trace:
        for offset in range(trace_len):
            prev = index - offset
            if prev < 0 or not ball_track[prev][0]:
                continue
            draw_x, draw_y = map(int, ball_track[prev])
            frame = cv2.circle(frame, (draw_x, draw_y), radius=3, color=(0, 255, 0), thickness=2)
    else:
        bx, by = map(int, ball_track[index])
        frame = cv2.circle(frame, (bx, by), radius=5, color=(0, 255, 0), thickness=2)
        frame = cv2.putText(frame, "ball", (bx + 8, by + 8), cv2.FONT_HERSHEY_SIMPLEX, 0.8,
                            (0, 255, 0), 2)
    return frame


def _draw_court_keypoints(frame, kps):
    if kps is None:
        return frame
    for kp in kps:
        x, y = int(kp[0, 0]), int(kp[0, 1])
        frame = cv2.circle(frame, (x, y), radius=0, color=(0, 0, 255), thickness=10)
    return frame


def _project_point(point, inv_mat, shape):
    pt = np.array(point, dtype=np.float32).reshape(1, 1, 2)
    pt = cv2.perspectiveTransform(pt, inv_mat)
    x = int(np.clip(pt[0, 0, 0], 0, shape[1] - 1))
    y = int(np.clip(pt[0, 0, 1], 0, shape[0] - 1))
    return x, y


def _blend_contour_map(base, heat_norm):
    if not np.any(heat_norm):
        return base.copy()
    heat_color = cv2.applyColorMap(heat_norm, cv2.COLORMAP_JET)
    overlay = base.copy()
    mask = heat_norm > 0
    overlay[mask] = cv2.addWeighted(heat_color[mask], HEAT_ALPHA, overlay[mask], 1 - HEAT_ALPHA, 0)
    for level in CONTOUR_LEVELS:
        _, thresh = cv2.threshold(heat_norm, level, 255, cv2.THRESH_BINARY)
        contours_info = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        contours = contours_info[0] if len(contours_info) == 2 else contours_info[1]
        if not contours:
            continue
        level_color = cv2.applyColorMap(np.full((1, 1), level, dtype=np.uint8), cv2.COLORMAP_JET)[0, 0]
        cv2.drawContours(overlay, contours, -1, tuple(int(c) for c in level_color), 2)
    return overlay


def _sample_positive_percentile(arr, percentile):
    """
    Estimate percentile from a strided subset to avoid allocating huge helper arrays.
    The stride is chosen so that the sampled view stays close to PERCENTILE_TARGET_SAMPLES.
    """
    if not np.any(arr > 0):
        return 0.0
    total = arr.shape[0] * arr.shape[1]
    stride = int(max(1, np.sqrt(total / (PERCENTILE_TARGET_SAMPLES + 1e-6))))
    sampled = arr[::stride, ::stride]
    positives = sampled[sampled > 0]
    if positives.size == 0:
        # Fallback to the entire array when the strided sample missed positives.
        positives = arr[arr > 0]
    return float(np.percentile(positives, percentile))


def combine_stream(frames, scenes, bounces, ball_track, homography_matrices, kps_court,
                   persons_top, persons_bottom, draw_trace=False, trace=10) -> Generator[CombineFrameOutputs, None, None]:
    """
    Generator variant that yields CombineFrameOutputs per frame to keep memory usage flat.
    """
    is_track = [mat is not None for mat in homography_matrices]

    for scene_start, scene_end in scenes:
        tracked = is_track[scene_start:scene_end]
        if not tracked or sum(tracked) / (len(tracked) + 1e-15) <= 0.5:
            neutral_court = get_court_img()
            if neutral_court.ndim == 2:
                neutral_court_color = cv2.cvtColor(neutral_court, cv2.COLOR_GRAY2BGR)
            else:
                neutral_court_color = neutral_court.copy()
            for frame in frames[scene_start:scene_end]:
                frame_copy = frame.copy()
                yield CombineFrameOutputs(
                    combined=frame_copy,
                    minimap_ball=neutral_court_color.copy(),
                    minimap_player=neutral_court_color.copy(),
                    heatmap_player=neutral_court_color.copy(),
                    heatmap_ball=neutral_court_color.copy(),
                )
            continue

        court_base = get_court_img()
        court_ball = court_base.copy()
        court_player_heat = get_court_img()
        court_ball_heat = get_court_img()
        heatmap_accum = np.zeros(court_base.shape[:2], dtype=np.float32)
        ball_heatmap_accum = np.zeros(court_base.shape[:2], dtype=np.float32)
        heat_target = PLAYER_HEAT_INCREMENT * 15
        ball_heat_target = BALL_HEAT_INCREMENT * 8

        for idx in range(scene_start, scene_end):
            frame = frames[idx].copy()
            inv_mat = homography_matrices[idx]
            frame = _draw_ball(frame, ball_track, idx, draw_trace, trace)
            frame = _draw_court_keypoints(frame, kps_court[idx] if idx < len(kps_court) else None)

            if idx in bounces and inv_mat is not None and ball_track[idx][0]:
                ball_point = _project_point(ball_track[idx], inv_mat, court_base.shape[:2])
                cv2.circle(court_base, ball_point, radius=0, color=(0, 255, 255), thickness=50)
                cv2.circle(court_ball, ball_point, radius=0, color=(0, 255, 255), thickness=50)
                ball_heatmap_accum = cv2.circle(ball_heatmap_accum, ball_point, BALL_HEAT_RADIUS,
                                                BALL_HEAT_INCREMENT, -1)

            minimap = court_base.copy()
            minimap_ball = court_ball.copy()
            if minimap_ball.ndim == 2:
                minimap_ball = cv2.cvtColor(minimap_ball, cv2.COLOR_GRAY2BGR)
            imgs_ball_frame = minimap_ball

            persons = persons_top[idx] + persons_bottom[idx]
            court_person = get_court_img()
            for bbox, person_point in persons:
                if len(bbox) == 0 or inv_mat is None:
                    continue
                x1, y1, x2, y2 = map(int, bbox)
                frame = cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
                px, py = _project_point(person_point, inv_mat, court_base.shape[:2])
                cv2.circle(minimap, (px, py), radius=0, color=(255, 0, 0), thickness=80)
                cv2.circle(court_person, (px, py), radius=0, color=(255, 0, 0), thickness=80)
                heatmap_accum = cv2.circle(heatmap_accum, (px, py), PLAYER_HEAT_RADIUS,
                                           PLAYER_HEAT_INCREMENT, -1)

            minimap_resized = cv2.resize(minimap, (MINIMAP_WIDTH, MINIMAP_HEIGHT))
            frame[30:30 + MINIMAP_HEIGHT, frame.shape[1] - 30 - MINIMAP_WIDTH:frame.shape[1] - 30] = minimap_resized
            combined_frame = frame

            if court_person.ndim == 2:
                court_person = cv2.cvtColor(court_person, cv2.COLOR_GRAY2BGR)
            imgs_person_frame = court_person

            heat_blurred = cv2.GaussianBlur(heatmap_accum, (0, 0),
                                            sigmaX=PLAYER_HEAT_GAUSSIAN_SIGMA,
                                            sigmaY=PLAYER_HEAT_GAUSSIAN_SIGMA)
            heat_percentile = _sample_positive_percentile(heat_blurred, PLAYER_HEAT_PERCENTILE)
            if heat_percentile == 0.0:
                imgs_heatmap_frame = court_player_heat.copy()
            else:
                heat_target = max(heat_target * PLAYER_HEAT_TARGET_DECAY,
                                  heat_percentile, PLAYER_HEAT_INCREMENT)
                heat_norm = np.clip((heat_blurred / (heat_target + 1e-6)) * 255.0, 0, 255).astype(np.uint8)
                imgs_heatmap_frame = _blend_contour_map(court_player_heat, heat_norm)

            ball_heat_blurred = cv2.GaussianBlur(ball_heatmap_accum, (0, 0),
                                                 sigmaX=BALL_HEAT_GAUSSIAN_SIGMA,
                                                 sigmaY=BALL_HEAT_GAUSSIAN_SIGMA)
            ball_percentile = _sample_positive_percentile(ball_heat_blurred, BALL_HEAT_PERCENTILE)
            if ball_percentile == 0.0:
                imgs_ball_heatmap_frame = court_ball_heat.copy()
            else:
                ball_heat_target = max(ball_heat_target * BALL_HEAT_TARGET_DECAY,
                                       ball_percentile, BALL_HEAT_INCREMENT)
                ball_heat_norm = np.clip((ball_heat_blurred / (ball_heat_target + 1e-6)) * 255.0,
                                         0, 255).astype(np.uint8)
                imgs_ball_heatmap_frame = _blend_contour_map(court_ball_heat, ball_heat_norm)

            yield CombineFrameOutputs(
                combined=combined_frame,
                minimap_ball=imgs_ball_frame,
                minimap_player=imgs_person_frame,
                heatmap_player=imgs_heatmap_frame,
                heatmap_ball=imgs_ball_heatmap_frame,
            )


def combine(*args, **kwargs):
    imgs_res, imgs_ball, imgs_person, imgs_heatmap, imgs_ball_heatmap = [], [], [], [], []
    for outputs in combine_stream(*args, **kwargs):
        imgs_res.append(outputs.combined)
        imgs_ball.append(outputs.minimap_ball)
        imgs_person.append(outputs.minimap_player)
        imgs_heatmap.append(outputs.heatmap_player)
        imgs_ball_heatmap.append(outputs.heatmap_ball)
    return imgs_res, imgs_ball, imgs_person, imgs_heatmap, imgs_ball_heatmap


In [3]:
# Combine into image
image_result, imgs_minimap_ball, imgs_minimap_player, imgs_heatmap_player, imgs_ball_heatmap = combine(frames,
                       scenes, 
                       bounces, 
                       ball_track, 
                       homography_matrices, 
                       kps_court, 
                       persons_top, 
                       persons_bottom,
                       draw_trace=True)

In [ ]:
convert_mp4 = False
# Save output video
# print("Saving output video...")
# write_video(image_result, fps, output_path, convert_mp4=convert_mp4)

# print("Saving minimap and heatmap images...")
# write_video(imgs_minimap_ball, fps, "results/minimap_ball", convert_mp4=convert_mp4)
# write_video(imgs_minimap_player, fps, "results/minimap_player", convert_mp4=convert_mp4)
# write_video(imgs_heatmap_player, fps, "results/heatmap_player", convert_mp4=convert_mp4)

print("Saving sample images...")
# write_image(imgs_minimap_ball, "results/", True, "minimap_ball")
# write_image(imgs_heatmap_player, "results/", True, "minimap_player")
write_image(imgs_ball_heatmap, "results/", True, "heatmap_ball")

Saving sample images...


In [5]:
stop_time = time.time()
print(f"Total processing time: {stop_time - start_time} seconds")

NameError: name 'time' is not defined